In [149]:
import pandas as pd
import datetime
import re
import time

In [150]:
# df1 = pd.read_csv('CSL.AX001.csv')
# df2 = pd.read_csv('CSL.AX002.csv')

In [151]:
df1 = pd.read_excel(r'..\New folder\WES.AX001.xlsx')
df2 = pd.read_excel(r'..\New folder\WES.AX002.xlsx')

Fill the empty Data Column with Previous Cell Value

In [152]:
df1['Date Posted'] = df1['Date Posted'].fillna(method='ffill')
df1 = df1.fillna('')

Converting into Datetime Object

In [153]:
df1['Date Posted'] = pd.to_datetime(df1['Date Posted'])
df2.Year = pd.to_datetime(df2.Year, format=r'%Y-%m-%d')

In [154]:
def remove_str(row):
    x = row.Comments2
    try:
        x = x.replace(re.search('Originally posted .*', x)[0], '').strip()
    except:
        pass
    try:
        x = x.replace(re.search('Release Date:.*', x)[0], '').strip()
    except:
        pass
    row.Comments2 = x
    return row
df1 = df1.apply(remove_str, axis='columns')

Previous Year's Number of Post

In [155]:
def prev_year(date_temp):
    return df1[(((df1['Date Posted'] - date_temp).astype('timedelta64[h]')/24) >= -365) & (((df1['Date Posted'] - date_temp).astype('timedelta64[h]')/24) <=0)].shape[0]

Total Post Related to Earnings

In [156]:
def earning_post(filtered_df, keyword):
    def func_inner(row):      
        for each_word in keyword:
            if each_word in str(row.Comments2):
                return 1
        return 0
    return sum(filtered_df.apply(func_inner, axis='columns'))

Remove Unwanted Lines from Comments

In [157]:
# filtered_df = filtered_df.apply(remove_str, axis='columns')

Row containing at least one negative words

In [158]:
def row_containing_neg_words(filtered_df, keyword):
    def func_inner(row):      
        for each_word in keyword:
            if each_word in str(row.Comments2):
                return 1
        return 0
    return sum(filtered_df.apply(func_inner, axis='columns'))

Number of Positive Words Mentioned

In [180]:
def positive_words(filtered_df, keyword):
    def func2(row):
        word_count = 0
        for each in keyword:
            word_count += len(re.findall(each, str(row.Comments2)))
        return word_count
    return sum(filtered_df.apply(func2, axis='columns'))

Number of Negative Words Mentioned

In [160]:
def criticism_words(filtered_df, keyword):
    def func2(row):
        word_count = 0
        for each in keyword:
            word_count += len(re.findall(each, str(row.Comments2)))
        return word_count
    return sum(filtered_df.apply(func2, axis='columns'))

Total Number of Words

In [161]:
def words_length(filtered_df):
    def func3(row):
        return len(str(row.Comments2).split())
    return sum(filtered_df.apply(func3, axis='columns'))

Sentiment

In [187]:
def sentiment(df):
    # df['Sentiment'] = df['Sentiment'].fillna('None')
    df.Sentiment = df.Sentiment.replace('', 'None')
    buy  = df.Sentiment.value_counts()['Buy']
    hold = df.Sentiment.value_counts()['Hold']
    sell = df.Sentiment.value_counts()['Sell']
    total = df.shape[0]
    sent = (buy+hold-sell)/total
    return sent



Output

In [188]:
def main_function(row): 
    pos_keyword = ['am', 'is', 'are', 'was', 'were']
    neg_keyword = ['am', 'is', 'are', 'was', 'were']
    earning = ['hi', 'hello']
    
    # Filtering Data Frame before 365 days
    filtered_df = df1[(((df1['Date Posted'] - row['Year']).astype('timedelta64[h]')/24) >= -365) & (((df1['Date Posted'] - row['Year']).astype('timedelta64[h]')/24) <=0)]
    row['Total Post in a Financial Year'] = filtered_df.shape[0]
    # row['Total post related to earnings'] = earning_post(filtered_df, earning)
    # Filtering Data that day, 1 day before and after
    filtered_df_3_days = df1[(abs((df1['Date Posted'] - row['Year']).astype('timedelta64[h]')/24) <= 1)]
    # Filtering data between 90 to 21 days
    filtered_df_90_21 = df1[(((df1['Date Posted'] - row['Year']).astype('timedelta64[h]')/24) >= -90) & (((df1['Date Posted'] - row['Year']).astype('timedelta64[h]')/24) <= -21)]
    row['Abnormal Discussions'] = filtered_df_3_days.shape[0]/filtered_df_90_21.shape[0]
    # row['Total number of positive words'] = positive_words(filtered_df_3_days, pos_keyword)
    # row['Criticism Comments'] = row_containing_neg_words(filtered_df_3_days, neg_keyword)
    # row['Total number of negative words'] = criticism_words(filtered_df_3_days, neg_keyword)
    # row['Total no. of words'] = words_length(filtered_df_3_days)
    row['Sentiment'] = sentiment(filtered_df_3_days)
    return row
df2 = df2.apply(main_function, axis='columns')

KeyError: 'Hold'

In [ ]:
sentiment(filtered_df_3_days)

NameError: name 'filtered_df_3_days' is not defined

In [ ]:
df2.to_csv('3.csv')

In [ ]:
df1.Sentiment = df1.Sentiment.fillna('None')
df1.Sentiment = df1.Sentiment.replace('', 'None')
buy = df1.Sentiment.value_counts()['Buy']
hold = df1.Sentiment.value_counts()['Hold']
sell = df1.Sentiment.value_counts()['Sell']
total = df1.shape[0]
sent = (buy+hold-sell)/total
sent

0.34827531394055

In [ ]:
df1.Sentiment.value_counts()

None    2945
Hold    1552
Buy      939
LT       481
Sell     300
ST        74
Name: Sentiment, dtype: int64